In [1]:
import csv
import os
import sys


PROJECT_ROOT = "/home/nlin/workspace/code/projects/autograd_cpp"
sys.path.append(PROJECT_ROOT)

In [2]:
def load_mnist_from_file(csvfile):
    reader = csv.reader(csvfile)

    data = []
    labels = []
    next(reader)
    for row in reader:
        row = [float(i) for i in row]
        data.append([i * 1.0 / 255 for i in row[1:]])
        labels.append([1 if row[0] == i else 0 for i in range(10)])
    
    n = len(data)
    csvfile.close()

    return data, labels, n

csvfile = open(os.path.join(PROJECT_ROOT, "data/mnist_test_short.csv"))
xs, ys, n = load_mnist_from_file(csvfile)

In [3]:
from learning.micrograd_engine import Value
from learning.micrograd_nn import Layer

nin = 28 * 28
W1 = Layer(nin, 256)
W2 = Layer(256, 10)

In [4]:
def softmax(x):
    """
    Inputs an array of 10 values, output scales them
    """
    assert len(x) == 10
    epsilon = 1e-4
    denom = sum([i.exp() for i in x], epsilon)
    return [i.exp() / denom for i in x]

def forward(x):
    Z1 = W1(x, relu=True)
    Z2 = softmax(W2(Z1, relu=False))
    return Z2

In [10]:
xs = xs[:5]
ys = ys[:5]

ypred = [forward(x) for x in xs]
loss = Value(0.0)

for y, yp in zip(ys, ypred):  
    loss += -sum([i * j.log() for i, j in zip(y, yp)])

print(loss)
loss.backward()

alpha = 0.01
params = [*W1.parameters(), *W2.parameters()]

for value in params:
    value.data -= alpha * value.grad
    value.grad = 0.0

new_ypred = [forward(x) for x in xs]
new_loss = Value(0.0)

for y, yp in zip(ys, new_ypred):  
    new_loss += -sum([i * j.log() for i, j in zip(y, yp)])

print(new_loss)

Value(data=1479.449667090684)
Value(data=694.2320520536119)
